Assignment 4:
- Below code is to run localy.
- It has all the four sections.
- In the fourth section, 4a has data generation
    - 4b has histogram ploting.

Also attaching python file used in the aws.
Out put files from each section is also attached.
AWS CLI is attached too.

In [ ]:
import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
import pyspark.sql as sql
from pyspark.sql.functions import sum as sum, col as col, when as when

donation_data_header_file = 'indiv_header_file.csv'
donation_data_file = 'itcont_2016_20161204_92060702.txt'

spark = sql.SparkSession.builder.master("local")\
        .appName("Assignment 4 spark session")\
        .getOrCreate()

donation_data_header_df = spark.read.option("header", "true") \
        .csv(donation_data_header_file)

donation_data_file_df = spark.read \
        .option("header", "false") \
        .option("sep", "|") \
        .csv(donation_data_file)

output = 'output'
donation_data_merged_df = donation_data_header_df.union(donation_data_file_df)

#Filter Data -  we are interested in these three campaigns.
def filter_data_on_campaign_id(donation_data_df):
    #C00575795 - HILLARY FOR AMERICA
    #C00577130 - Bernie Sanders committee (BERNIE)
    #C00580100 - DONALD J. TRUMP FOR PRESIDENT, INC
    campaign_commite_id_list = ['C00575795', 'C00577130', 'C00580100']
    return donation_data_df[donation_data_df['CMTE_ID'].isin(campaign_commite_id_list)]

donation_data_df = filter_data_on_campaign_id(donation_data_merged_df)

# 1. How many donations did each campaign have?
def number_of_donations_per_campaign(donation_data_df):
    # Filter negative amounts. Assuming this as return of donations.
    donation_data_df = donation_data_df.filter(donation_data_df['TRANSACTION_AMT'] > 0)
    number_of_donations_grouped = donation_data_df.groupBy('CMTE_ID').count()
    number_of_donations_grouped.coalesce(1).write.mode('overwrite').format('csv') \
        .option('header', 'true') \
        .save(output+'/'+'number_of_donations_per_campaign')
    return number_of_donations_grouped

number_of_donations_data = number_of_donations_per_campaign(donation_data_df)
number_of_donations_data.show()

# 2. What was the total amount donated to each campaign?
def donation_amount_per_campaign(donation_data_df):
    # Considering the negative numbers as well. This is helpfull in calculating final donation amount.
    donation_amount = donation_data_df.groupBy('CMTE_ID').agg(sum('TRANSACTION_AMT').alias("TOTAL_DONATION"))
    donation_amount.coalesce(1).write.mode('overwrite').format('csv') \
        .option('header', 'true') \
        .save(output+'/'+'donation_amount_per_campaign')
    return donation_amount

donation_amount_per_campaign_data = donation_amount_per_campaign(donation_data_df)
donation_amount_per_campaign_data.show()

# 3. What percentage of the each campaigns donations was done by small contributors?
def small_donation_percentage(donation_data_df):
    # Filter negative donations
    positive_donations_df = donation_data_df.filter(donation_data_df['TRANSACTION_AMT'] > 0)
    # Filter small donations and find the percentage
    small_donations_percentage_df = positive_donations_df.groupBy('CMTE_ID') \
        .agg(((sum(when(col('TRANSACTION_AMT') < 200, \
                        col('TRANSACTION_AMT'))) / sum(col('TRANSACTION_AMT'))) * 100).alias("SMALL_DONATIONS_PERCENTAGE"))
    small_donations_percentage_df.coalesce(1).write.mode('overwrite').format('csv')\
        .option('header', 'true') \
        .save(output+'/'+'small_donations_percentage')
    return small_donations_percentage_df

small_donations_detail = small_donation_percentage(donation_data_df)
small_donations_detail.show()

# 4a. Produce a histogram of the donations for each campaign?
# For this we will be downloading the data from aws and plotting locally. Data generation method in AWS.
donation_data_df = donation_data_df.select('CMTE_ID', 'TRANSACTION_AMT', 'NAME')
donation_data_df.coalesce(1).write.mode('overwrite').format('csv') \
    .option('header', 'true') \
    .save(output+'/'+'donation_data_for_histogram')

# 4b. Produce a histogram of the donations for each campaign? Run it locally#
donations_data_from_aws = 'number_of_donations_from_aws.csv'
donations_data_from_aws_df = pd.read_csv(donations_data_from_aws)

hillary_data = donations_data_from_aws_df[(donations_data_from_aws_df['CMTE_ID'] == 'C00575795')]
bernie_data = donations_data_from_aws_df[(donations_data_from_aws_df['CMTE_ID'] == 'C00577130')]
trump_data = donations_data_from_aws_df[(donations_data_from_aws_df['CMTE_ID'] == 'C00580100')]

plt.figure(figsize=(15,4))
sb.distplot(trump_data['TRANSACTION_AMT'])
plt.xlabel("Donations")
plt.ylabel("Frequency")
plt.title("Histogram Plot for donations for Trump campaign")
plt.show()

plt.figure(figsize=(15,4))
sb.distplot(hillary_data['TRANSACTION_AMT'])
plt.xlabel("Donations")
plt.ylabel("Frequency")
plt.title("Histogram Plot for donations for Bernie campaign")
plt.show()

plt.figure(figsize=(15,4))
sb.distplot(bernie_data['TRANSACTION_AMT'])
plt.xlabel("Donations")
plt.ylabel("Frequency")
plt.title("Histogram Plot for donations for Hillary campaign")
plt.show()

spark.stop()